In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import time
import pandas as pd

In [ ]:
fiis_list = ["snag11"]

In [ ]:
fiis_indicators = []
for fii in fiis_list:
  url = f"https://statusinvest.com.br/fiagros/{fii}"
  
  time.sleep(0.25)
  try:
    req = Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36')
    html = urlopen(req)
    soup = BeautifulSoup(html, 'html.parser')
      
    fund = soup.find("div", id="fund-section")
    main = soup.find("main", id="main-2")

    name = fund.find_all("div", "info")[1].find("strong").getText()
    init = fund.find_all("div", "info")[2].find("strong").getText()
    sector = fund.find_all("div", "info")[5].find("strong").getText()
    type = fund.find_all("div", "info")[4].find("strong").getText()
    administrator = fund.find("div", "container").find("strong", "fw-700").getText()

    price = main.find("div", title="Valor atual do ativo").find("strong").getText()
    dividend_yield = main.find("div", title="Dividend Yield com base nos últimos 12 meses").find("strong").getText()
    net_worth = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[0].find("span", "sub-value").getText().replace("R$ ","")
    asset_value = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[0].find("strong").getText()
    pvp = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[1].find("strong").getText()
    shareholders = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[5].find("strong").getText()
    lastDiv = main.find("div", "mb-5 d-flex flex-wrap flex-lg-nowrap justify-between").find_all("div", recursive=False)[1].find("strong").getText()
    nextDiv = main.find("div", "mb-5 d-flex flex-wrap flex-lg-nowrap justify-between").find_all("div", recursive=False)[2].find("strong").getText()

    dict = {  "fii": fii,
        "nome": name,
        "segmento": sector,
        "tipo": type,
        "administrador": administrator,
        "cotistas" : shareholders,
        "patrimonio": net_worth,
        "vp" : asset_value,
        "valor_atual": price,
        "pvp": pvp,
        "dividend_yield": dividend_yield,
        "ultimo dividendo" : lastDiv,
        "proximo dividendo" : nextDiv,
    }
    fiis_indicators.append(dict)
  except Exception as e:
    print(e)
    print(e.stacktrace)

In [ ]:
print(fiis_indicators)

In [ ]:
df = pd.DataFrame.from_dict(fiis_indicators)

In [ ]:
df.to_excel("fiagros.xlsx", index=False)